In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd


page = requests.get("http://www.bahntrassenradeln.de/bahn__radindex.htm")
page.content[:1000]

b'<html><head>\n<title>Bahntrassenradeln - \xdcbersicht mach Bewertung [Achim Bartoschek]</title>\n<link rel="stylesheet" type="text/css" media="screen" href="include/b.css">\n<link rel="stylesheet" type="text/css" media="print" href="include/d.css">\n<script src="include/b.js" type="text/javascript"></script>\n<meta http-equiv="content-type" content="text/html; charset=iso-8859-1">\n<meta name="robots" content="index, follow">\n<script src="include/jquery-1.11.3.min.js" type="text/javascript"></script>\n<script src="include/j.js" type="text/javascript"></script>\n<script type="text/javascript">\n strA      = "4"; strL = "ri"; strS = ""; strP = "";\n</script>\n</head>\n\n<body>\n\n<!--Kopf-->\n<script type="text/javascript">mK();</script>\n<noscript>Homepage von Dr. Achim Bartoschek &#150; Bitte aktivieren Sie JavaScript.</noscript>\n\n<!--Titel-->\n<table class="b" align="center"><tr>\n <td class="mb"><img class="mini" src="fotos/grafik/k_de.png"></td>\n <td class="mn">&nbsp;</td>\n <

In [2]:
soup = bs(page.content)

In [5]:
bewertungsstufen=soup.find_all('h2')[1:]
bewertungsstufen

[<h2>Bewertung mit 4 Rädern</h2>,
 <h2>Bewertung mit 3 Rädern</h2>,
 <h2>Bewertung mit 2 Rädern</h2>,
 <h2>Bewertung mit einem Rad</h2>,
 <h2>Bewertung mit 0 Rädern</h2>,
 <h2>Noch nicht bewertet</h2>]

In [7]:

df_all=pd.DataFrame()

for i,dummy in enumerate(bewertungsstufen):
    data = str(soup.find_all(class_="st")[i])

    df = pd.DataFrame([x.split('</a>') for x in data.split('•')], 
                      columns=['id','name'])[1:]

    df['bewertungsstufe']=str(bewertungsstufen[i])
    df_all=df_all.append(df, ignore_index=True)
df_all.head()

,id,name,bewertungsstufe
0,"<a href=""bahn_bw.htm#bw2_01"">BW 2.01",Wanderbahn: Mudau  Mosbach<br/>\n,<h2>Bewertung mit 4 Rädern</h2>
1,"<a href=""bahn_bw.htm#bw2_08"">BW 2.08",Radfernweg Stauferland: Göppingen-Faurndau  ...,<h2>Bewertung mit 4 Rädern</h2>
2,"<a href=""bahn_bw.htm#bw2_13"">BW 2.13",Reutlingen  Lichtenstein (ehem. Zahnradbahn)...,<h2>Bewertung mit 4 Rädern</h2>
3,"<a href=""bahn_by.htm#by1_01a"">BY 1.01a",Rhönexpress-Bahnradweg: Zeitlofs  Wildflecke...,<h2>Bewertung mit 4 Rädern</h2>
4,"<a href=""bahn_by.htm#by1_07"">BY 1.07",Gaubahn-Radwanderweg: Ochsenfurt  Röttingen ...,<h2>Bewertung mit 4 Rädern</h2>


In [8]:
df_all['link']=df_all.id.str.split('">').str[0].str.replace('<a href="','')

In [9]:
df_all['id']=df_all.id.str.split('">').str[1]

In [10]:
df_all['name']=df_all.name.str.replace('<br/>\n','').str.replace('</p>','')
df_all['bewertungsstufe']=df_all.bewertungsstufe.str.replace('<h2>','').str.replace('</h2>','')

In [11]:
df_all

,id,name,bewertungsstufe,link
0,BW 2.01,Wanderbahn: Mudau  Mosbach,Bewertung mit 4 Rädern,bahn_bw.htm#bw2_01
1,BW 2.08,Radfernweg Stauferland: Göppingen-Faurndau  ...,Bewertung mit 4 Rädern,bahn_bw.htm#bw2_08
2,BW 2.13,Reutlingen  Lichtenstein (ehem. Zahnradbahn),Bewertung mit 4 Rädern,bahn_bw.htm#bw2_13
3,BY 1.01a,Rhönexpress-Bahnradweg: Zeitlofs  Wildflecken,Bewertung mit 4 Rädern,bahn_by.htm#by1_01a
4,BY 1.07,Gaubahn-Radwanderweg: Ochsenfurt  Röttingen ...,Bewertung mit 4 Rädern,bahn_by.htm#by1_07
...,...,...,...,...
783,SN 3.06,Goßdorf-Kohlmühle  Lohsdorf  Hohnstein (in ...,Noch nicht bewertet,bahn_sn.htm#sn3_06
784,ST 1.02a,Hohenwulsch  Bismark,Noch nicht bewertet,bahn_st.htm#st1_02a
785,ST 1.13,Gernrode  Rieder  Ballenstedt,Noch nicht bewertet,bahn_st.htm#st1_13
786,TH24,Rudolstadt-Schwarza  Bad Blankenburg,Noch nicht bewertet,bahn_th.htm#th24


In [102]:
df_all.to_csv('alle_bew.csv', index=False)

In [101]:
df_all[df_all.id.str.contains('ST')]

,id,name,bewertungsstufe,link
173,ST 2.10,Zuckerbahn-Radweg: Zeitz  Camburg,Bewertung mit 3 Rädern,bahn_st.htm#st2_10
397,ST 1.01,Salzwedel  Steinitz,Bewertung mit 2 Rädern,bahn_st.htm#st1_01
398,ST 1.03,Gardelegen Nord  Wernstedt,Bewertung mit 2 Rädern,bahn_st.htm#st1_03
399,ST 1.04,Goldbeck  Werben (Elbe),Bewertung mit 2 Rädern,bahn_st.htm#st1_04
400,ST 1.05,Neuermark-Lübars  Sandau (Elbe),Bewertung mit 2 Rädern,bahn_st.htm#st1_05
401,ST 1.05a,Genthin  Jerichow,Bewertung mit 2 Rädern,bahn_st.htm#st1_05a
402,ST 1.08,ehem. Anschlussbahn Magdeburg-Buckau,Bewertung mit 2 Rädern,bahn_st.htm#st1_08
403,ST 1.09,Magdeburg-Cracau  Biederitz,Bewertung mit 2 Rädern,bahn_st.htm#st1_09
404,ST 1.11,Burg  Grabow und Küsel  Magdeburgerforth  ...,Bewertung mit 2 Rädern,bahn_st.htm#st1_11
405,ST 1.16,Vogelsdorf  Dingelstedt,Bewertung mit 2 Rädern,bahn_st.htm#st1_16
